In [1]:
import pandas as pd
import os
import datetime
from collections import Counter
import numpy as np

### Dataframes

In [2]:
df_tickets = pd.read_excel(f"DTICKETHISTAB_BETON.xlsx",engine='openpyxl')

In [3]:
df_orders = pd.read_excel(f"DORDERSTAB_BETON.xlsx",sheet_name='data',engine='openpyxl')

In [4]:
tickets_cp = df_tickets.copy(deep=True)

### Matrice de temps

In [5]:
df_temps = pd.read_csv(f"Matrice_de_temps_all.csv",header=None)

In [6]:
temps = df_temps.to_numpy()

### Localisation des dépots

In [7]:
df_depots = pd.read_excel(f"adresses_complete.xlsx",engine='openpyxl')
df_depots = df_depots[df_depots['Location_type'] != 'customer']
df_depots.dropna(axis=1,  inplace=True)
df_depots['NBR'] = df_depots['JOB_DESC'].apply(
    lambda x: x if len(x.split(' ')) == 1 else int(x.split(' ')[1]))
df_depots.set_index('NBR',inplace=True)
df_depots.drop(['Location_type', 'JOB_DESC', 'MAP_PAGE', 
       'Postal_code', 'JOBSTATE', 'LATITUDE_Y', 'LONGITUDE_X'],axis=1,inplace=True)

In [9]:
def beginTime(week,day,driver):
    week_op = df_tickets[df_tickets['WEEK']==week]
    day_op = week_op[week_op['DATE']==day]
    _data = day_op[day_op['DRIVER_NBR']==driver][["BEGIN_LOAD","ARRIVE_PLANT"]]
    _data.sort_values(["BEGIN_LOAD"],inplace=True)
   
    return f"{_data.iloc[0,0].time()}".split('.')[0]

def endTime(week,day,driver):
    week_op = df_tickets[df_tickets['WEEK']==week]
    day_op = week_op[week_op['DATE']==day]
    _data = day_op[day_op['DRIVER_NBR']==driver][["BEGIN_LOAD","ARRIVE_PLANT"]]
    _data.sort_values(["BEGIN_LOAD"],inplace=True)
    return f"{_data.iloc[-1,1].time()}".split('.')[0]

def setDriverLocId(driver):
    _df=df_tickets[df_tickets['DRIVER_NBR']==driver] 
    return Counter(_df['SHIP_LOC']).most_common(1)[0][0]

In [10]:
tickets = tickets_cp.copy(deep=True)
print(len(tickets))
tickets.dropna(axis=0,inplace=True)
print(len(tickets))
tickets['DATE'] = tickets['TICKET_DATE'].apply(lambda x:datetime.date(x.year,x.month,x.day))
tickets['WEEK'] = tickets['DATE'].apply(lambda x:datetime.date(x.year,x.month,x.day).isocalendar()[1])
tickets['DAY'] = tickets['DATE'].apply(lambda x:datetime.date(x.year,x.month,x.day).toordinal())
tickets = tickets[tickets['SHIP_LOC']!=99]
tickets = tickets[tickets['PROD_NBR']=='Z']
tickets.drop(['DTICKETHIS_TICKET_ID',  'TICKET_NBR', 'TICKET_DATE',
       'TICKET_TIME',  'PROD_NBR',  'CATEGORY_CODE','DTICKETPRODUCTHIS_TICKET_ID'],axis=1,inplace=True)
tickets['BEGIN_LOAD'] = tickets['BEGIN_LOAD'].apply(lambda x:x.hour*60+x.minute)
tickets['BEGIN_POUR'] = tickets['BEGIN_POUR'].apply(lambda x:x.hour*60+x.minute)
tickets['FINISH_POUR'] = tickets['FINISH_POUR'].apply(lambda x:x.hour*60+x.minute)
tickets['ARRIVE_PLANT'] = tickets['ARRIVE_PLANT'].apply(lambda x:x.hour*60+x.minute)
tickets['FINISH_LOAD'] = tickets['FINISH_LOAD'].apply(lambda x:x.hour*60+x.minute)
tickets['TO_PLANT'] = tickets['TO_PLANT'].apply(lambda x:x.hour*60+x.minute)
tickets['ON_JOB'] = tickets['ON_JOB'].apply(lambda x:x.hour*60+x.minute)
tickets['TO_JOB'] = tickets['TO_JOB'].apply(lambda x:x.hour*60+x.minute)

tickets.head()


108304
107795


,SHIP_LOC,ORDER_ID,DRIVER_NBR,BEGIN_LOAD,BEGIN_POUR,FINISH_POUR,ARRIVE_PLANT,TRUCK_NBR,SCHED_LOC,FINISH_LOAD,TO_PLANT,ON_JOB,TO_JOB,LOAD,LOADNR,QUANTITY,DATE,WEEK,DAY
4,54,917231,15083.0,856,900,933,994,460,54,856,933,900,866,1,1,8.0,2020-10-01,40,737699
10,54,917231,15143.0,900,959,964,994,370,54,866,980,939,900,2,1,8.0,2020-10-01,40,737699
16,54,917231,15070.0,920,978,1005,1030,426,54,920,1005,978,939,3,1,8.0,2020-10-01,40,737699
22,54,917231,15507.0,959,1101,1101,1101,384,54,959,1101,1009,978,4,1,8.0,2020-10-01,40,737699
28,54,917231,7000059.0,975,1069,1083,1115,019,54,978,1111,994,986,7,1,7.0,2020-10-01,40,737699


In [11]:
sched = tickets[["DATE","WEEK","DAY","SHIP_LOC","SCHED_LOC","ORDER_ID","DRIVER_NBR","TRUCK_NBR","BEGIN_LOAD","FINISH_LOAD",
                  "TO_JOB","ON_JOB","BEGIN_POUR","FINISH_POUR","TO_PLANT","ARRIVE_PLANT","QUANTITY","LOAD","LOADNR"]]

In [41]:
sched.reset_index(inplace=True)
sched.set_index(["DATE"],inplace=True)
test=sched.loc[datetime.date(2020, 11, 28)]
list_drivers = list(np.unique(test['DRIVER_NBR']))
list_orders = list(np.unique(test['ORDER_ID']))

 Pour chaque chauffeur, trouver l'ordre des visites de la journée, loading at depot, unloading at customer

In [36]:
test.reset_index(inplace=True)
test.set_index(['ORDER_ID'],inplace=True)
print(f"{len(np.unique(test['DRIVER_NBR']))}")
for i, order in enumerate(list_orders):
    print(f"***Ordre N {i} {order} ", np.unique(test.loc[order]["SHIP_LOC"]), np.unique(test.loc[order]["SCHED_LOC"]),
          f"   Nb Driver {len(np.unique(test.loc[order]['DRIVER_NBR']))}")
    try:
        print( f"   Nb visite {len(test.loc[order].to_numpy()[:,0])}")
    except IndexError:
        print(f"   Nb visite 1")
    print(f"   {np.unique(test.loc[order]['DRIVER_NBR'])}")
    
        

27
***Ordre N 0 933897  [54] [54]    Nb Driver 14
   Nb visite 22
   [  14347.   15107.   15243.   15405.   15422.   15428.   15467.   15562.
   15598.   15607.   15614. 7000032. 7000038. 7000052.]
***Ordre N 1 935940  [54] [54]    Nb Driver 9
   Nb visite 9
   [  15107.   15161.   15405.   15467.   15562.   15598.   15607. 7000038.
 7000052.]
***Ordre N 2 935979  [58] [58]    Nb Driver 4
   Nb visite 32
   [1.000e+00 2.019e+03 2.057e+03 1.556e+04]
***Ordre N 3 936678  [10] [10]    Nb Driver 1
   Nb visite 1
   [15369.]
***Ordre N 4 936746  [10] [10]    Nb Driver 2
   Nb visite 2
   [  15423. 7000066.]
***Ordre N 5 937483  [54] [54]    Nb Driver 9
   Nb visite 14
   [14347. 15107. 15161. 15243. 15405. 15422. 15428. 15467. 15614.]
***Ordre N 6 937588  [10] [10]    Nb Driver 2
   Nb visite 2
   [  137. 15083.]
***Ordre N 7 937836  [10] [10]    Nb Driver 8
   Nb visite 14
   [4.000000e+00 1.370000e+02 1.508300e+04 1.536900e+04 1.542300e+04
 1.548800e+04 7.000029e+06 7.000066e+06]


### le chauffeur 1 est utilisé pour l'ordre 935979. Verifier dans le ficier Excel et identifier pourquoi je ne l'ai pas ignoré dans mes instances

In [39]:
t93=test.loc[935979]
t93

,DATE,index,WEEK,DAY,SHIP_LOC,SCHED_LOC,DRIVER_NBR,TRUCK_NBR,BEGIN_LOAD,FINISH_LOAD,TO_JOB,ON_JOB,BEGIN_POUR,FINISH_POUR,TO_PLANT,ARRIVE_PLANT,QUANTITY,LOAD,LOADNR
ORDER_ID,,,,,,,,,,,,,,,,,,,
935979,2020-11-28,105659,48,737757,58,58,2019.0,448,404,404,412,420,427,450,458,484,8.0,1,1
935979,2020-11-28,105666,48,737757,58,58,2057.0,529,412,412,422,428,451,470,470,479,8.0,2,1
935979,2020-11-28,105673,48,737757,58,58,15560.0,528,421,421,430,438,464,470,478,502,8.0,3,1
935979,2020-11-28,105680,48,737757,58,58,1.0,6401 BEL,434,434,448,460,470,484,502,502,12.0,4,1
935979,2020-11-28,105687,48,737757,58,58,1.0,6406 BEL,448,448,460,470,478,503,503,510,12.0,5,1
935979,2020-11-28,105694,48,737757,58,58,1.0,6402 BEL,460,460,467,484,503,507,514,517,12.0,6,1
935979,2020-11-28,105701,48,737757,58,58,1.0,103VAIL,475,475,477,492,507,514,521,525,7.5,7,1
935979,2020-11-28,105708,48,737757,58,58,1.0,706NEPVE,484,484,491,503,514,544,544,557,11.0,8,1
935979,2020-11-28,105715,48,737757,58,58,2057.0,529,493,493,499,506,544,544,560,588,8.0,9,1


In [94]:
print("Demande ",sum(test.loc[933897]["QUANTITY"]))
t93["dToC"]=t93["ON_JOB"]-t93["TO_JOB"]
t93["cToD"]=t93["ARRIVE_PLANT"]-t93["TO_PLANT"]
t93["load"]=t93["FINISH_LOAD"]-t93["BEGIN_LOAD"]
t93["pour"]=t93["FINISH_POUR"]-t93["BEGIN_POUR"]
t93


Demande  156.0


/home/alious/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/alious/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/alious/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

,DATE,WEEK,DAY,SHIP_LOC,SCHED_LOC,DRIVER_NBR,TRUCK_NBR,BEGIN_LOAD,FINISH_LOAD,TO_JOB,...,FINISH_POUR,TO_PLANT,ARRIVE_PLANT,QUANTITY,LOAD,LOADNR,dToC,cToD,load,pour
ORDER_ID,,,,,,,,,,,,,,,,,,,,,
933897,2020-11-28,48,737757,54,54,15107.0,367,482,508,508,...,508,508,526,1.0,1,1,0,18,26,0
933897,2020-11-28,48,737757,54,54,15243.0,449,452,452,457,...,569,571,586,6.0,2,1,11,15,0,75
933897,2020-11-28,48,737757,54,54,7000052.0,090,462,462,479,...,581,585,585,8.0,3,1,15,0,0,80
933897,2020-11-28,48,737757,54,54,15422.0,321,490,490,501,...,587,587,589,8.0,4,1,5,2,0,53
933897,2020-11-28,48,737757,54,54,15428.0,362,502,502,508,...,588,593,595,8.0,5,1,6,2,0,1
933897,2020-11-28,48,737757,54,54,14347.0,469,514,514,525,...,607,613,613,8.0,6,1,19,0,0,19
933897,2020-11-28,48,737757,54,54,7000038.0,157,534,534,554,...,618,618,618,5.5,7,1,27,0,0,11
933897,2020-11-28,48,737757,54,54,15614.0,345,568,568,582,...,647,660,660,8.0,8,2,25,0,0,29
933897,2020-11-28,48,737757,54,54,7000032.0,246,584,584,597,...,661,664,695,4.0,9,2,16,31,0,14


In [44]:
#%% Clean data
df_tickets.dropna(axis=0,inplace=True)
df_tickets['DATE'] = df_tickets['TICKET_DATE'].apply(lambda x:datetime.date(x.year,x.month,x.day))
df_tickets['WEEK'] = df_tickets['DATE'].apply(lambda x:datetime.date(x.year,x.month,x.day).isocalendar()[1])
df_tickets['DAY'] = df_tickets['DATE'].apply(lambda x:datetime.date(x.year,x.month,x.day).toordinal())

df_tickets = df_tickets[df_tickets['SHIP_LOC']!=99]
df_tickets = df_tickets[df_tickets['PROD_NBR']=='Z']

df_tickets.drop(['DTICKETHIS_TICKET_ID',  'TICKET_NBR', 'TICKET_DATE',
       'TICKET_TIME',   'BEGIN_POUR',
       'FINISH_POUR',   'FINISH_LOAD',
       'TO_PLANT', 'ON_JOB', 'TO_JOB', 'LOAD', 'LOADNR',
       'DTICKETPRODUCTHIS_TICKET_ID', 'PROD_NBR',  'CATEGORY_CODE'],axis=1,inplace=True)



In [45]:
df_tickets["BEGIN"]=df_tickets.apply(lambda x: beginTime(x.WEEK,x.DATE,x.DRIVER_NBR),axis=1)

,SHIP_LOC,ORDER_ID,DRIVER_NBR,BEGIN_LOAD,ARRIVE_PLANT,TRUCK_NBR,SCHED_LOC,QUANTITY,DATE,WEEK,DAY,BEGIN,END
4,54,917231,15083.0,2020-10-01 14:16:23.720,2020-10-01 16:34:16.832,460,54,8.0,2020-10-01,40,737699,07:38:00,16:34:16
10,54,917231,15143.0,2020-10-01 15:00:56.091,2020-10-01 16:34:14.566,370,54,8.0,2020-10-01,40,737699,07:24:38,16:34:14
16,54,917231,15070.0,2020-10-01 15:20:11.886,2020-10-01 17:10:50.250,426,54,8.0,2020-10-01,40,737699,06:57:28,17:10:50
22,54,917231,15507.0,2020-10-01 15:59:26.468,2020-10-01 18:21:08.017,384,54,8.0,2020-10-01,40,737699,06:46:49,18:21:08
28,54,917231,7000059.0,2020-10-01 16:15:50.000,2020-10-01 18:35:24.000,019,54,7.0,2020-10-01,40,737699,07:05:27,18:35:24


In [ ]:
df_tickets["END"]=df_tickets.apply(lambda x: endTime(x.WEEK,x.DATE,x.DRIVER_NBR),axis=1)

In [46]:
df_tickets["DRIVER_LOC"]=df_tickets.apply(lambda x: setDriverLocId(x.DRIVER_NBR),axis=1)

In [47]:
df_tickets.head()

,SHIP_LOC,ORDER_ID,DRIVER_NBR,BEGIN_LOAD,ARRIVE_PLANT,TRUCK_NBR,SCHED_LOC,QUANTITY,DATE,WEEK,DAY,BEGIN,END,DRIVER_LOC
4,54,917231,15083.0,2020-10-01 14:16:23.720,2020-10-01 16:34:16.832,460,54,8.0,2020-10-01,40,737699,07:38:00,16:34:16,10
10,54,917231,15143.0,2020-10-01 15:00:56.091,2020-10-01 16:34:14.566,370,54,8.0,2020-10-01,40,737699,07:24:38,16:34:14,20
16,54,917231,15070.0,2020-10-01 15:20:11.886,2020-10-01 17:10:50.250,426,54,8.0,2020-10-01,40,737699,06:57:28,17:10:50,20
22,54,917231,15507.0,2020-10-01 15:59:26.468,2020-10-01 18:21:08.017,384,54,8.0,2020-10-01,40,737699,06:46:49,18:21:08,54
28,54,917231,7000059.0,2020-10-01 16:15:50.000,2020-10-01 18:35:24.000,019,54,7.0,2020-10-01,40,737699,07:05:27,18:35:24,54


In [48]:
df_orders['DATE'] = df_orders['SHIPDATE'].apply(lambda x:datetime.date(x.year,x.month,x.day))
df_orders['WEEK'] = df_orders['DATE'].apply(lambda x:datetime.date(x.year,x.month,x.day).isocalendar()[1])
df_orders['DAY'] = df_orders['DATE'].apply(lambda x:datetime.date(x.year,x.month,x.day).toordinal())
df_orders.drop(['COMP_NBR',  'ORDER_STATUS',
       'ORDER_DATE', 'ORDER_TIME (min)', 'SHIPDATE', 'Day_of_the_week',
       'SHIPTIME (min)', 'CUST_NBR', 'ZONE_NBR', 'WORK_TYPE', 'SCHED_LOC',
       'JOB_DESC', 'ORDER_DESC', 'MAP_PAGE', 'JOBCITY', 'Postal code ',
       'JOBSTATE', 'LATITUDE_Y', 'LONGITUDE_X', 'TRAVEL_MINUTES',
       'UNLOAD_MINUTES', 'QTY_PER_HOUR', 'TRUCK_REQUEST', 'PROD_NBR',
       'CATEGORY_CODE', 'UNIT_OF_MEASURE', 'QUANTITY', 'LOOKUP_DESC',
       'CONFIG_DESC', 'Unnamed: 33', 'Unnamed: 34', 'Location ',
       'ID','DATE','WEEK','DAY'],axis=1,inplace=True)
df_orders.drop_duplicates(inplace=True)

In [49]:
df_tickets.reset_index()
df=pd.merge(left=df_tickets,right= df_orders,how="inner",on='ORDER_ID')
# df = df[df['DRIVER_NBR']!=1]
df.head()

,SHIP_LOC,ORDER_ID,DRIVER_NBR,BEGIN_LOAD,ARRIVE_PLANT,TRUCK_NBR,SCHED_LOC,QUANTITY,DATE,WEEK,DAY,BEGIN,END,DRIVER_LOC,ORDER_LOC,ORDER_NBR,LOCATION ID
0,54,917231,15083.0,2020-10-01 14:16:23.720,2020-10-01 16:34:16.832,460,54,8.0,2020-10-01,40,737699,07:38:00,16:34:16,10,0,603592,0
1,54,917231,15143.0,2020-10-01 15:00:56.091,2020-10-01 16:34:14.566,370,54,8.0,2020-10-01,40,737699,07:24:38,16:34:14,20,0,603592,0
2,54,917231,15070.0,2020-10-01 15:20:11.886,2020-10-01 17:10:50.250,426,54,8.0,2020-10-01,40,737699,06:57:28,17:10:50,20,0,603592,0
3,54,917231,15507.0,2020-10-01 15:59:26.468,2020-10-01 18:21:08.017,384,54,8.0,2020-10-01,40,737699,06:46:49,18:21:08,54,0,603592,0
4,54,917231,7000059.0,2020-10-01 16:15:50.000,2020-10-01 18:35:24.000,019,54,7.0,2020-10-01,40,737699,07:05:27,18:35:24,54,0,603592,0


### Ajouter Id  dans la matrice de distance des dépots

In [50]:
df['SHIP_LOC_ID'] = df.apply(lambda x: df_depots.loc[x.SHIP_LOC]['Location ID'],axis=1)
df['SCHED_LOC_ID'] = df.apply(lambda x: df_depots.loc[x.SCHED_LOC]['Location ID'],axis=1)
df['DRIVER_LOC_ID'] = df.apply(lambda x: df_depots.loc[x.DRIVER_LOC]['Location ID'],axis=1)

In [51]:
df.head()

,SHIP_LOC,ORDER_ID,DRIVER_NBR,BEGIN_LOAD,ARRIVE_PLANT,TRUCK_NBR,SCHED_LOC,QUANTITY,DATE,WEEK,DAY,BEGIN,END,DRIVER_LOC,ORDER_LOC,ORDER_NBR,LOCATION ID,SHIP_LOC_ID,SCHED_LOC_ID,DRIVER_LOC_ID
0,54,917231,15083.0,2020-10-01 14:16:23.720,2020-10-01 16:34:16.832,460,54,8.0,2020-10-01,40,737699,07:38:00,16:34:16,10,0,603592,0,2494,2494,2490
1,54,917231,15143.0,2020-10-01 15:00:56.091,2020-10-01 16:34:14.566,370,54,8.0,2020-10-01,40,737699,07:24:38,16:34:14,20,0,603592,0,2494,2494,2491
2,54,917231,15070.0,2020-10-01 15:20:11.886,2020-10-01 17:10:50.250,426,54,8.0,2020-10-01,40,737699,06:57:28,17:10:50,20,0,603592,0,2494,2494,2491
3,54,917231,15507.0,2020-10-01 15:59:26.468,2020-10-01 18:21:08.017,384,54,8.0,2020-10-01,40,737699,06:46:49,18:21:08,54,0,603592,0,2494,2494,2494
4,54,917231,7000059.0,2020-10-01 16:15:50.000,2020-10-01 18:35:24.000,019,54,7.0,2020-10-01,40,737699,07:05:27,18:35:24,54,0,603592,0,2494,2494,2494


### Ajouter le temps parcouru aller-retour par visite

In [52]:
df['t1']=df.apply(lambda x: (temps[x.SHIP_LOC_ID][x['LOCATION ID']]+temps[x['LOCATION ID']][x.SHIP_LOC_ID])/60,axis=1)
df['t2']=df.apply(lambda x: (temps[x.SCHED_LOC_ID][x['LOCATION ID']]+temps[x['LOCATION ID']][x.SCHED_LOC_ID])/60,axis=1)

In [53]:
df.reset_index(inplace=True)
df.set_index(['WEEK'],inplace=True)
df.sort_index(inplace=True)
df = df.loc[[41, 43, 44, 45, 46, 47, 48]]

### Calculer la distance parcourue par jour/ par jour et par chauffeur

In [54]:
test=df.copy(deep=True)
list_date=np.unique(test["DATE"])
my_dict = dict()
day_dict=dict()
driver_dict=dict()
count=0
count1=0
for day in list_date:
    op = test[test['DATE']==day]
    list_drivers = np.unique(op["DRIVER_NBR"])
    som_day=0
    for i,driver_nbr in enumerate(list_drivers):
        if driver_nbr==999:
            continue
        driver_op = op[op['DRIVER_NBR']==driver_nbr]
        locations = list(driver_op[["SHIP_LOC_ID","LOCATION ID"]].to_numpy().flatten())
        driverLoc=list(driver_op['DRIVER_LOC_ID'])[0]
        locations.append(driverLoc)
        locations.insert(0,driverLoc)
        prev=0
        somme=0
        for nex in range(1,len(locations)):
            somme+=temps[locations[prev]][locations[nex]]
            prev=nex
        
        my_dict[count] = [ f"{day}",f"{driver_nbr}",f"{somme/60:2.2f}",driverLoc]
        som_day =som_day+somme
        count=count+1
#         print(driver_nbr,' ',day,' ',)
#         print(locations)
#         print(driver_op[[]])
        
#         break
#     break
    day_dict[count1] = [ f"{day}",f"{som_day/60:2.2f}"]
    count1=count1+1
pd.DataFrame.from_dict(my_dict, orient='index',columns=['date','driver','distance','driverLoc']).to_csv('drivers_distances.csv')
pd.DataFrame.from_dict(day_dict, orient='index',columns=['date','distance']).to_csv('days_distances.csv')


### Distance par jour/ par ordre/ par chauffeur

In [50]:
test=df.copy(deep=True)
list_date=np.unique(test["DATE"])

my_dict = dict()
day_dict=dict()
driver_dict=dict()
count=0
count1=0
list_date=[datetime.date(2020,10,10)]
for day in list_date:
    op_ = test[test['DATE']==day]
    list_orders = np.unique(op_["ORDER_ID"])
    
    for order in list_orders:
        som_day=0
        op=op_[op_["ORDER_ID"]==order]
        list_drivers = np.unique(op["DRIVER_NBR"])
        
        for i,driver_nbr in enumerate(list_drivers):
            if driver_nbr==999:
                continue
            driver_op = op[op['DRIVER_NBR']==driver_nbr]
            locations = list(driver_op[["SHIP_LOC_ID","LOCATION ID"]].to_numpy().flatten())
            driverLoc=list(driver_op['DRIVER_LOC_ID'])[0]
            locations.append(driverLoc)
            locations.insert(0,driverLoc)
            prev=0
            somme=0
            print(f"Ordre {order} driver {driver_nbr}")
            for nex in range(1,len(locations)):
                print(f"Temps de {locations[prev]} à {locations[nex]} = {temps[locations[prev]][locations[nex]]/60:2.2f} ")
                somme=somme+temps[locations[prev]][locations[nex]]
                prev=nex

            my_dict[count] = [ f"{day}",f"{order}",f"{driver_nbr}",f"{somme/60:2.2f}",driverLoc]
#             print(my_dict[count])
            som_day =som_day+somme
            count=count+1
    #         print(driver_nbr,' ',day,' ',)
    #         print(locations)
    #         print(driver_op[[]])

    #         break
    #     break
    day_dict[count1] = [ f"{day}",f"{som_day/60:2.2f}"]
    count1=count1+1
# pd.DataFrame.from_dict(my_dict, orient='index',columns=['date','order','driver','distance','driverLoc']).to_csv('orders_distances.csv')
# pd.DataFrame.from_dict(day_dict, orient='index',columns=['date','distance']).to_csv('days_distances.csv')


Ordre 925444 driver 15398.0
Temps de 2494 à 2494 = 0.00 
Temps de 2494 à 68 = 22.67 
Temps de 68 à 2494 = 22.15 
Ordre 925444 driver 15467.0
Temps de 2494 à 2494 = 0.00 
Temps de 2494 à 68 = 22.67 
Temps de 68 à 2494 = 22.15 
Ordre 927089 driver 15431.0
Temps de 2491 à 2494 = 18.17 
Temps de 2494 à 617 = 18.86 
Temps de 617 à 2491 = 19.32 
Ordre 927569 driver 15161.0
Temps de 2491 à 2494 = 18.17 
Temps de 2494 à 146 = 6.60 
Temps de 146 à 2494 = 5.32 
Temps de 2494 à 146 = 6.60 
Temps de 146 à 2491 = 19.18 
Ordre 927569 driver 15398.0
Temps de 2494 à 2494 = 0.00 
Temps de 2494 à 146 = 6.60 
Temps de 146 à 2494 = 5.32 
Temps de 2494 à 146 = 6.60 
Temps de 146 à 2494 = 5.32 
Temps de 2494 à 146 = 6.60 
Temps de 146 à 2494 = 5.32 
Ordre 927569 driver 15405.0
Temps de 2489 à 2494 = 26.42 
Temps de 2494 à 146 = 6.60 
Temps de 146 à 2489 = 26.45 
Ordre 927569 driver 15422.0
Temps de 2489 à 2494 = 26.42 
Temps de 2494 à 146 = 6.60 
Temps de 146 à 2494 = 5.32 
Temps de 2494 à 146 = 6.60 
Temps

In [93]:
pd.DataFrame.from_dict(my_dict, orient='index',)

,date,driver,distance
0,2020-10-01,2001.0,121.01
1,2020-10-01,2006.0,260.71
2,2020-10-01,2019.0,197.26
3,2020-10-01,2023.0,224.26
4,2020-10-01,2026.0,169.95
...,...,...,...
5231,2020-11-30,7000062.0,77.77
5232,2020-11-30,7000063.0,122.64
5233,2020-11-30,7000065.0,185.41
5234,2020-11-30,7000066.0,222.99


In [84]:
a=[1,2,3,4]
prev=0
som=0
for nex in range(1,len(a)):
    print(prev,' ',nex)
    prev=nex
som

0   1
1   2
2   3


0

In [16]:
test=df.reset_index(inplace=True)
test.set_index(['DATE','DRIVER_NBR'],inplace=True)
test.sort_index(inplace=True)
np.unique(test['DATE'])

AttributeError: 'NoneType' object has no attribute 'set_index'

In [155]:
# data with distance
df.reset_index(inplace=True)
df.set_index(['WEEK','DATE','DRIVER_NBR'],inplace=True)
df.sort_index(inplace=True)
result=df.groupby(by=['WEEK','DATE','DRIVER_NBR','BEGIN','END']).agg({'t1':'sum','t2':'sum','QUANTITY':'sum'})
result.to_csv('schedule_dist_quant.csv')
result.head()

t1          t2  QUANTITY
WEEK DATE       DRIVER_NBR BEGIN    END                                       
41   2020-10-05 999.0      14:21:41 14:21:41   87.268333   87.268333       4.5
                2001.0     06:12:37 16:26:11  196.638333  196.638333      36.0
                2006.0     06:20:00 15:23:13  194.711667  194.711667      34.0
                2019.0     06:24:51 16:37:06  254.880000  254.880000      35.0
                2026.0     06:41:28 16:23:37  139.581667  139.581667      46.0

In [17]:
df.groupby(by=['DATE','WEEK','ORDER_ID']).agg({'t1':'sum','t2':'sum'})

t1          t2
DATE       WEEK ORDER_ID                        
2020-10-05 41   917234     12.705000   12.705000
                920229    128.483333  128.483333
                920230    154.208333  154.208333
                921407    147.800000  147.800000
                922129    613.525000  613.525000
...                              ...         ...
2020-11-28 48   936678     32.168333   32.168333
                936746     78.166667   78.166667
                937483    139.813333  139.813333
                937588    128.330000  128.330000
                937836    439.226667  439.226667

[3290 rows x 2 columns]

In [18]:
df.groupby(by=['DATE','DRIVER_NBR','ORDER_ID']).agg({'t1':'sum','t2':'sum'})


t1          t2
DATE       DRIVER_NBR ORDER_ID                        
2020-10-05 2001.0     923252     82.733333   82.733333
                      923521     33.505000   33.505000
                      924646     80.400000   80.400000
           2006.0     922496    247.050000  247.050000
                      925256     41.935000   41.935000
...                                    ...         ...
2020-11-28 7000038.0  935940     41.338333   41.338333
           7000052.0  933897     30.465000   30.465000
                      935940     41.338333   41.338333
           7000066.0  936746     39.083333   39.083333
                      937836     31.373333   31.373333

[11209 rows x 2 columns]

In [19]:
t=df.groupby(by=['DATE','WEEK','DRIVER_NBR']).agg({'t1':'sum','t2':'sum'})
# t.to_csv('driver.csv')
# t.xs(('2020-11-16',47)).sum()

In [20]:
d= df.groupby(by=['DATE','ORDER_ID','DRIVER_NBR']).agg({'t1':'sum','t2':'sum'})
# d.xs(("2020-10-30"))
print(d.xs((930040),level=1)/60)
print(d.xs((930040),level=1).sum()/60)

# df['DATE'].unique()

                             t1        t2
DATE       DRIVER_NBR                    
2020-10-31 15107.0     0.349583  0.349583
           7000029.0   0.349583  0.349583
           7000031.0   0.349583  0.349583
t1    1.04875
t2    1.04875
dtype: float64
